# Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks  

## Abstract  

最先进的物体检测网络依靠区域提议算法来推测物体的位置。SPPnet 和 Fast R-CNN 等进展减少了这些检测网络的运行时间，使得区域提议计算成为瓶颈。在这项工作中，我们引入了一个区域提议网络（RPN），该网络与检测网络共享全图像卷积特征，从而实现了几乎免费的区域提议。RPN 是一个全卷积网络，可同时预测每个位置的对象边界和对象分数。RPN 经过端到端训练以生成高质量区域提议，Fast R-CNN 将其用于检测。我们将 RPN 和 Fast R-CNN 通过共享其卷积特征进一步合并为一个网络-使用最近流行的具有“注意”机制的神经网络术语，RPN 组件告诉统一网络在哪里寻找。对于非常深的 VGG-16 模型，我们的检测系统在 GPU 上的帧速率为 5fps（包括所有步骤），同时在 PASCAL VOC 2007,2012 和 MS COCO 数据集上实现了最先进的目标检测精度，每张图片只有300个提议。在 ILSVRC 和 COCO 2015 比赛中，Faster R-CNN 和 RPN 是多个比赛任务中获得第一名的基础。代码已公开发布。  

## 1 INTRODUCTION  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/1.jpg?raw=true)

目标检测的最新进展是由区域提议方法（例如[4]）和基于区域的卷积神经网络（RCNN）[5]的成功所驱动的。尽管[5]中最初开发的基于区域的 CNN 的计算量很大，但由于在各种提议中共享卷积，因此其成本已大幅降低。最新的版本 Fast R-CNN [2]使用非常深的网络实现接近实时的速率，而忽略了花在区域提议上的时间。现在，提议是最先进的检测系统中测试时间的计算瓶颈。  

区域建议方法通常依赖廉价特征和经济推断方案。Selective Search [4]是最流行的方法之一，它贪婪地合并基于工程低级特征的超级像素。然而，与有效的检测网络[2]相比，选择性搜索速度慢了一个数量级，在 CPU 实现中每个图像 2 秒。EdgeBoxes[6]目前在提议质量和速度之间提供了最佳平衡，每张图像只有 0.2 秒。尽管如此，区域提议步骤仍然会消耗与检测网络一样多的运行时间。  

有人可能会注意到，fast region-based CNN 利用了 GPU，而在研究中使用的区域提议方法在 CPU 上实现，使得这种运行时比较不公平。加速提议计算的一个显而易见的方法是将其重新实现为 GPU。这可能是一个有效的工程解决方案，但重新实施忽略了下游检测网络，因此错过了共享计算的重要机会。  

在本文中，我们展示了一个算法改变计算提议与深卷积神经网络-导致一个优雅和有效的解决方案，提议计算几乎免费计算。为此，我们引入了新颖的区域提议网络（RPNs），它们共享对象检测网络的卷积层。通过在测试时共享卷积，计算提议的边际成本很小（例如，每个图像10ms）。  

我们的观察结果是基于区域的检测器所使用的卷积特征映射（如Fast RCNN）也可用于生成区域提议。在这些卷积特征之上，我们通过添加一些额外的卷积层来构建 RPN，这些卷积层同时在规则网格上的每个位置处回归区域边界和对象分数。RPN 因此是一种全卷积网络（FCN），并且可以专门用于生成检测提议的任务。  

RPN 旨在有效预测各种比例和纵横比的区域提议。与使用图像金字塔（图1，a）或过滤器金字塔（图1，b）的流行方法不同，我们引入 “anchor” boxes
 作为多种比例和纵横比的参考。我们的方案可以被认为是回归参考金字塔（图1，c），它避免了枚举多个比例或长宽比的图像或过滤器。该模型在使用单一尺度图像进行训练和测试时表现良好，从而有利于运行速度。  

为了将 RPN 与 Fast R-CNN [2]物体检测网络统一起来，我们提出了一种训练方案，该方案在区域提议任务的微调和微调物体检测之间进行交替，同时保持提议的固定。该方案迅速收敛并产生具有卷积特征的统一网络，这些特征在两个任务之间共享。  

我们在 PASCAL VOC 检测基准[11]中综合评估了我们的方法，其中使用 Fast R-CNN 的 RPN 产生的检测精度优于使用 Fast R-CNN 的选择性搜索强基线。同时，我们的方法在测试时几乎免除了选择性搜索的所有计算负担 - 提议的有效运行时间仅为 10 毫秒。使用[3]的昂贵的非常深的模型，我们的检测方法在 GPU 上仍然具有 5fps 的帧率（包括所有步骤），因此在速度和准确性方面都是实用的对象检测系统。我们还报告了 MS COCO 数据集[12]的结果，并使用 COCO 数据研究了对 PASCAL VOC 的改进。代码已经在https://github.com/shaoqingren/faster_rcnn （MATLAB）和 https://github.com/rbgirshick/py-faster-rcnn （Python）上公开发布。  

这篇手稿的初稿已经在之前发表[10]。从那时起，RPN 和 Faster R-CNN 的框架已被采用并推广到其他方法，如 3D 对象检测[13]，part-based 检测[14]，实例分割[15]和图像字幕[16]。我们的快速和有效的物体检测系统也已建立在商业系统中，如 Pinterests，并报告了用户参与度的提升。  

在 ILSVRC 和 COCO 2015 比赛中，Faster R-CNN 和 RPN 是 ImageNet 检测，ImageNet 定位，COCO 检测和 COCO 分割任务中的几个第一名[18]的基础。 RPN 完全学习从数据中提出区域，因此可以从更深入和更具表现力的特征（例如[18]中采用的 101 层残差网络）中轻松获益。Faster R-CNN 和 RPN 也被这些比赛中的其他几个主要参赛者使用。这些结果表明，我们的方法不仅是实际使用的经济高效的解决方案，而且是提高目标检测精度的有效方法。  

## 2 RELATED WORK  

**Object Proposals** 有关对象提议方法有大量文献。目标提议方法的全面调查和比较可以在[19]，[20]，[21]中找到。广泛使用的对象提议方法包括基于超像素分组（例如 Selective Search [4]，CPMC [22]，MCG [23]）和基于滑动窗口的方法（例如，objectness in windows[24]，EdgeBoxes[6]）。对象建议方法被采用为独立于检测器（例如，选择性搜索[4]物体检测器，RCNN [5]和 Fast R-CNN[2]）的外部模块。  

**Deep Networks for Object Detection** R-CNN方法[5]端到端地训练 CNN 以将提议区域分类为对象类别或背景。R-CNN 主要用作分类器，它不能预测对象边界（除了通过边界框回归进行修正）。其准确度取决于区域提议模块的性能（请参见[20]中的比较）。有几篇论文提出了使用深度网络来预测对象边界框的方法。在 OverFeat 方法[9]中，训练全连接层以预测假定单个对象定位任务的框坐标。然后将全连接的层变成卷积层以检测多个特定类别的对象。MultiBox 方法[26]，[27]从一个网络生成区域提议，该网络的最后一个全连接层同时预测多个不同类别的盒子，从而推广 OverFeat 的“单盒子”方式。这些与类别不相关的盒子被用作 R-CNN 的提议[5]。与我们的全卷积方案相比，MultiBox 提议网络适用于单幅图像 crop 或多幅大型图像crop（例如224×224）。MultiBox 不共享提议和检测网络之间的特征。稍后在我们的方法的上下文中，我们将更深入地讨论 OverFeat 和 MultiBox。与我们的工作同时，DeepMask 方法[28]是为学习分割提议而开发的。  

卷积的共享计算已经越来越受到人们越来越多的关注，从而获得高效而准确的视觉识别。OverFeat 论文[9]计算来自图像金字塔的卷积特征用于分类，定位和检测。为了有效的基于区域的对象检测[1]，[30]和语义分割[29]，开发了共享卷积特征映射上的自适应大小的 pooling（SPP）[1]。Fast R-CNN [2]能够对共享卷积特征进行端到端检测器训练，并显示出令人信服的准确性和速度。  

## 3 FASTER R-CNN  

我们的对象检测系统称为 Faster R-CNN，由两个模块组成。第一个模块是提出区域的深度全卷积网络，第二个模块是使用提出的区域的 Fast R-CNN 检测器[2]。整个系统是用于物体检测的单一统一网络（图2）。RPN 模块使用最近流行的具有'注意力'[31]机制的神经网络术语，告诉 Fast R-CNN 模块在哪里寻找。在第 3.1 节中，我们介绍区域建议网络的设计和属性。在第 3.2 节中，我们开发了用于训练具有共享特征模块的算法。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/2.jpg?raw=true)

### 3.1 Region Proposal Networks  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/3.jpg?raw=true)

一个区域提议网络（RPN）以任意大小的图像作为输入，并输出一组矩形对象提议，每个提议都有一个对象评分。我们用全卷积网络对这个过程进行建模[7]。因为我们的最终目标是与 Fast R-CNN 对象检测网络共享计算[2]，所以我们假设两个网络共享一组共同的卷积层。在我们的实验中，我们研究了具有5 个可共享卷积层的 Zeiler and Fergus 模型[32]（ZF）和具有 13 个可共享卷积层的 Simonyan and Zisserman 模型[3]（VGG-16）。  

为了生成区域提议，我们在由最后的共享卷积层输出的卷积特征映射上滑动一个小网络。这个小网络将输入卷积特征映射的 n×n 空间窗口作为输入。每个滑动窗口被映射到一个较低维度的特征（ZF 为256-d，VGG为512-d）。这个特征被送入两个全连接层 - 一个盒子回归图层（reg）和一个盒子分类图层（cls）。我们在本文中使用 n = 3，注意到输入图像上的有效接受场很大（ZF和VGG分别为171和228像素）。这个迷你网络在图3（左）的单个位置进行说明。请注意，因为迷你网络以滑动窗口的方式运行，所有空间位置共享全连层。这种架构自然地用一个 n×n 卷积层，然后是两个1×1卷积层（分别用于reg和cls）来实现。  

#### 3.1.1 Anchors  

在每个滑动窗口位置，我们同时预测多个区域提议，其中每个位置的最大可能提议的数量被表示为 k。因此，reg 层具有 4k 个输出，编码 k 个框的坐标，并且 cls 层输出 2k 个评分，其估计每个提议的对象或不对象的概率。k 个提议相对于 k 个参考框被参数化，我们称之为锚。锚点位于所讨论的滑动窗口的中心，并且与比例和纵横比相关（图3左）。默认情况下，我们使用 3 个比例和 3 个高宽比，在每个滑动位置产生 k = 9 个锚点。对于大小为W×H（通常约为2400）的卷积特征映射，总共有 WHk 个锚。  

##### Translation-Invariant Anchors  

我们的方法的一个重要特性是它是 translates 不变的，无论是在锚点还是计算提议相对于锚点的函数方面。如果在图像中 translates 对象，则应该 translates 提议，并且相同的函数应该能够在任一位置预测提议。这种平移不变的特性由我们的方法保证。作为比较，MultiBox 方法[27]使用 k-means 生成 800 个锚点，这不是平移不变量。因此，如果 translates 对象，MultiBox 不保证会生成相同的提议。  

平移不变属性也减小了模型的大小。MultiBox 有一个（4 + 1）×800 维全连接输出层，而我们的方法在 k = 9 个锚点的情况下有一个（4 + 2）×9 维卷积输出层。因此，我们的输出层具有 $2.8×10^4$ 个参数（对于VGG-16为512×（4 + 2）×9），比具有 $6.1×10^6$ 参数（1536 ×(4 + 1)GoogleNet）的 MultiBox 输出层少两个数量级。如果考虑特征投影层，我们的提议层仍然比 MultiBox 少一个数量级的参数。我们认为我们的方法在小数据集上过拟合的风险较低，如 PASCAL VOC。  

##### Multi-Scale Anchors as Regression References  

我们的锚点设计提出了一种解决多尺度（和纵横比）的新方案。如图1 所示，多尺度预测有两种流行的方法。第一种方法是基于图像/特征金字塔，例如在DPM [8]和基于 CNN 的方法中。图像被调整多尺度大小，并且针对每个尺度（图1（a））计算特征图（HOG [8]或深卷积特征）。这种方式通常很有用，但是非常耗时。第二种方法是在特征图上使用多个比例（和/或宽高比）的滑动窗口。例如，在DPM [8]中，使用不同的滤波器尺寸（例如5×7和7×5）分别对不同长宽比的模型进行训练。如果这种方式用于解决多个尺度问题，则可以将其视为“过滤器金字塔”（图1（b））。第二种方式通常与第一种方式共同采用[8]。  

作为比较，我们的基于锚的方法建立在 pyramid of anchors 上，这是更具成本效益的。我们的方法参考多个比例和长宽比的锚箱来分类和回归边界框。 它只依赖单一尺度的图像和特征图，并使用单一尺寸的 filter（特征地图上的滑动窗口）。我们通过实验展示了这种方案对多种尺度和尺寸的效果（表8）。   

由于这种基于锚点的多尺度设计，我们可以简单地使用在单尺度图像上计算的卷积特征，像 Fast R-CNN 检测器中的一样。多尺度锚的设计是共享特征的关键组件，不需要额外的成本来解决尺度问题。  

#### 3.1.2 Loss Function  

为了训练 RPN，我们为每个锚点分配一个二进制类标签（是或不是目标）。 我们为两种锚点分配一个正向标签：（i）与真实标注具有最高 Intersection-over-Union（IoU）的锚点，或者（ii）与任何真实标注具有高于 0.7 的 IoU 重叠的锚点。请注意，单个真实框可以为多个锚点分配肯定的标签。通常第二个条件足以确定正样本；但我们仍然采用第一个条件，原因是在少数情况下，第二个条件可能找不到正样本。如果锚点以所有真实边界框的 IoU 比率低于 0.3，就认为该锚点是负样本。既不正面也不负面的锚不会有助于训练目标。  

通过这些定义，我们将 Fast R-CNN 中的多任务损失的目标函数最小化[2]。我们对图像的损失函数定义为：  

$L(\{p_i\},\{t_i\}) = \frac{1}{N_{cls}} \sum_i L_{cls}(p_i,p_i^*) + \lambda \frac{1}{N_{reg}} \sum_i p_i^* L_{reg}(t_i,t_i^*) \tag{1}$

在这里，i 是一个小批量中的锚点的索引，$p_i$ 是 i 作为对象的预测概率。如果锚为正，则真实标签 $p^*_i$ 为 1，如果锚为负，则为0。$t_i$ 是表示预测边界框的 4 个参数化坐标的矢量，并且 $t^*_i$ 是与正锚点相关联的真实框的矢量。分类损失 Lcls 是两个类（对象与非对象）的对数损失。对于回归损失，我们使用 $L_{reg}(t_i,t_i^*) = R(t_i − t_i^*)$ 其中 R 是[2]中定义的鲁棒损失函数（smooth L1）。术语 $p_i^* L_{reg}$ 表示回归损失仅在正锚 $(p_i^*=1)$ 时被激活，否则被禁用 $(p_i^*=0)$。cls 和 reg 层的输出分别由{pi}和{ti}组成。  

这两项由 $N_{cls}$ 和 $N_{reg}$ 进行归一化，并由平衡参数 $\lambda$ 加权。在我们当前的实现中（如在发布的代码中），方程(1)中的 cls 项通过mini-batch 规模（即，Ncls=256）进行归一化，并且 reg 项通过锚点位置的数量（即，Nreg〜2,400）进行归一化。默认情况下，我们设λ= 10，因此 cls和 reg 项的权重大致相等。我们通过实验表明，结果在很宽的范围内对 λ 的值不敏感（表9）。 我们还注意到，如上所述的标准化不是必需的，可以简化。 

对于边界框回归，我们采用[5]之后的4个坐标的参数化：  

$
\begin{align}
t_x = (x-x_a)/w_a, \ t_y = (y-y_a)/h_a, \\
t_w = \log(w/w_a), \ t_w = = \log(h/h_a), \\
t_x^* = (x^*-x_a)/w_a, \ t_y^* = (y^*-y_a)/h_a, \\
t_w^* = \log(w^*/w_a), \ t_h^* = = \log(h^*/h_a),
\end{align} \tag{2}
$

其中 x，y，w 和 h 表示框的中心坐标及其宽度和高度。变量 $x,x_a$ 和 $x^*$ 分别预测框，锚点框和 ground-truth 框（y，w，h 也一样）。这可以被认为是从锚箱到附近的真实边界框回归。  

尽管如此，我们的方法通过与之前的 RoI-based（Region of Interest）方法不同的方式实现了边界框回归。在[1]，[2]中，对从任意大小的 RoI 池化的特征执行边界框回归，并且回归权重由所有区域大小共享。在我们的公式中，用于回归的特征在特征图上具有相同的空间大小（3×3）。为了说明不同的尺寸，学习一组 k 个边界框回归器。每个回归者负责一个尺度和一个纵横比，而 k 个回归者不共享权重。因此，由于锚的设计，即使特征具有固定的尺寸/比例，仍然可以预测各种尺寸的边框。  
 
#### 3.1.3 Training RPNs  

RPN 可以通过反向传播和随机梯度下降（SGD）进行端对端训练[35]。我们遵循[2]中的“以图像为中心”的采样策略来训练这个网络。每个小批量产生于包含许多正面和负面示例锚点的单个图像。可以针对所有锚点的损失函数进行优化，但是由于它们占主导地位，这将偏向于负样本。相反，我们在图像中随机采样 256 个锚点以计算小批量的损失函数，其中采样的正锚和负锚具有高达 1：1 的比率。如果图像中少于 128 个正面样本，我们使用负样本填充mini-batch。  

我们通过从标准偏差为 0.01 的零均值高斯分布中绘制权重来随机初始化所有新图层。所有其他层（即共享卷积层）通过预训练用于 ImageNet 分类的模型[36]来初始化，如同标准实践[5]一样。我们调整 ZF 网络的所有层。对于 60k 小批量，我们使用 0.001 的学习率，PASCAL VOC 数据集中接下来的 20k 小批量使用 0.0001。我们使用 0.9 的动量和 0.0005 的重量衰减[37]。我们使用 Caffe[38]实现。  

### 3.2 Sharing Features for RPN and Fast R-CNN  

到目前为止，我们已经描述了如何训练用于区域提议生成的网络，而没有考虑在 region-based object detection CNN 中使用这些提议。对于检测网络，我们采用 Fast R-CNN [2]。接下来，我们将介绍用共享卷积层学习由 RPN 和 Fast R-CNN 组成的统一网络的算法（图2）。  

独立训练的 RPN 和 Fast R-CNN 将以不同的方式修改卷积层。因此，我们需要开发一种允许在两个网络之间共享卷积层的技术，而不是学习两个单独的网络。我们讨论了特征共享的三种训练网络的方法：  

（i）交替训练。在这个解决方案中，我们首先训练 RPN，并使用这些提议来训练 Fast R-CNN。由 Fast R-CNN 微调的网络然后用于初始化 RPN，并且该过程被重复。这是本文所有实验中使用的解决方案。  

（二）近似的联合训练。在这个解决方案中，RPN 和 Fast R-CNN 网络在训练期间合并为一个网络，如图2 所示。在每次 SGD 迭代中，正向传递生成区域提议，这些提议在训练 Fast R-CNN 检测器时像之前单独训练 Fast R-CNN 时一样。反向传播像往常一样发生，其中对于共享层，来自 RPN 损失和 Fast R-CNN 损失的反向传播信号被组合。这个解决方案很容易实现。但是这个解决方案忽略了导数 w.r.t.提议框的坐标也是网络响应，也是近似的。在我们的实验中，我们凭经验发现这个解决方案产生了接近的结果，但与交替训练相比，训练时间减少了约 25-50%。这个求解器包含在我们发布的 Python 代码中。  

（三）非近似联合训练。如上所述，由 RPN 预测的边界框也是输入的函数。Fast R-CNN 中的 RoI pooling 层[2]接受卷积特征并将预测的边界框作为输入，所以理论上有效的反向传播求解器也应该包含梯度 w.r.t.箱子坐标。在上述近似联合训练中，这些梯度被忽略。在一个非近似的联合训练解决方案中，我们需要一个可区分 wi.t. 的 RoI 池化层。这是一个非凡问题，可以通过[15]中开发的“RoI warping”层给出解决方案，这超出了本文的范围。  

**四步交替训练** 在本文中，我们采用实用的 4 步训练算法通过交替优化学习共享特征。在第一步中，我们按照 3.1.3 节的描述训练 RPN。该网络使用ImageNet 预先训练的模型进行初始化，并针对区域提议任务进行端到端微调。在第二步中，我们使用由步骤1 RPN 生成的提议，由 Fast R-CNN 训练单独的检测网络。该检测网络也由 ImageNet 预先训练的模型初始化。此时两个网络不共享卷积层。在第三步中，我们使用检测器网络初始化 RPN 训练，但是我们修复了共享卷积层，并且只对 RPN 特有的层进行了微调。现在这两个网络共享卷积层。最后，保持共享卷积层的固定，我们对 Fast R-CNN 的特有的层进行微调。因此，两个网络共享相同的卷积层并形成统一的网络。类似的交替训练可以运行更多的迭代，但我们观察到可以忽略的改进。  

### 3.3 Implementation Details  

我们在单一尺度的图像上训练和测试区域提议和对象检测网络。我们重新缩放图像，使它们的短边为 s = 600像素[2]。多尺度特征提取（使用图像金字塔）可能会提高精度，但不会表现出良好的速度精度折衷[2]。在重新缩放的图像上，最后一个卷积层上的 ZF 和 VGG 网络的总步幅为16个像素，因此在调整大小（〜500×375）之前，典型的 PASCAL 图像上的约 10 个像素。即使如此大的步幅也能提供良好的效果，但步幅更小，精确度可能会进一步提高。  

对于锚点，我们使用 3 个比例，盒子面积分别为 1282,2562 和 5122 像素，以及 3：1：1，1：2 和 2：1 的高宽比。这些超参数不是针对特定数据集仔细选择的，我们将在下一节中提供有关其影响的消融实验。如前所述，我们的解决方案不需要图像金字塔或 filter 金字塔来预测多个比例的区域，从而节省大量运行时间。图3（右）显示了我们的方法在广泛的尺度和宽高比方面的能力。表1 显示了使用 ZF 网络的每个锚的学习平均 proposal 大小。我们注意到，我们的算法允许预测值大于潜在感受域。这样的预测并非不可能 - 如果只有对象的中间部分是可见的，那么仍然可以大致推断出对象的范围。  

跨越图像边界的锚箱需要小心处理。在训练期间，我们忽略了所有的跨界锚，所以他们不会造成损失。对于典型的 1000×600 图像，总共将有大约 20000 个（≈60×40×9）锚点。在忽略跨界锚点的情况下，每个图像有大约 6000 个锚点用于训练。如果跨界异常值在训练中不被忽略，它们会在目标中引入大的，难以纠正的错误项，并且训练不会收敛。然而，在测试过程中，我们仍然将全卷积 RPN 应用于整个图像。这可能会生成跨边界 proposal，我们将其剪切到图像边界。  

一些 RPN 提议高度重叠。为了减少冗余，我们根据他们的 cls 分数对提议区域采用非极大值抑制（NMS）。我们将 NMS 的IoU阈值修正为 0.7，这使得我们每个图像约有 2000 个提议区域。正如我们将要展示的那样，NMS 不会损害最终检测的准确性，但会大大减少提议的数量。在 NMS 之后，我们使用排名top-N 提议区域进行检测。接下来，我们使用 2000 个 RPN 提议对 Fast R-CNN 进行训练，但在测试时评估不同数量的提议。  

## 4 EXPERIMENTS  

### 4.1 Experiments on PASCAL VOC  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/12.jpg?raw=true)

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/4.jpg?raw=true)

我们全面评估了我们在 PASCAL VOC 2007 检测基准[11]中的方法。该数据集包含 20 个对象类别的大约 5k 个训练图像和 5k 个测试图像。我们还为几个模型提供 PASCAL VOC 2012 基准测试结果。对于 ImageNet 预训练网络，我们使用具有 5 个卷积层和 3 个全连接层的 ZF net [32]的“fast”版本，以及具有 13 个卷积层和 3 个全连接层的公共 VGG-16 model[3]。我们主要评估检测 mean Average Precision（mAP），因为这是检测对象的实际指标（而不是关注对象提议代理指标）。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/6.jpg?raw=true)

表2（顶部）显示了使用各种区域建议方法进行训练和测试时的 Fast R-CNN 结果。这些结果使用 ZF 网络。对于选择性搜索（SS）[4]，我们通过“快速”模式生成约 2000 个提议。对于 EdgeBoxes（EB）[6]，我们通过调整 0.7 IoU 的默认 EB 设置生成提议。在 Fast R-CNN 框架下，SS 拥有 58.7% 的mAP，EB 拥有 58.6% 的 mAP。采用 RPN 的 Fast R-CNN 实现了有竞争力的结果，使用 300 个提议时，mAP 为59.9%。由于共享卷积计算，使用 RPN 产生比使用 SS 或 EB 更快的检测系统；较少的提议也会减少 region-wise 全连接层的成本（表5）。  

**RPN 上的消融实验** 为了研究 RPNs 作为提议方法的行为，我们进行了几项消融研究。首先，我们展示在 RPN 和 Fast R-CNN 检测网络之间共享卷积层的效果。为此，我们在四步培训过程的第二步之后停止。使用单独的网络将结果略微降低至58.7%（RPN + ZF，非共享，表2）。我们观察到这是因为在第三步中，当使用 detector tuned 特征来微调 RPN 时，提议质量得到改善。  

接下来，我们分析了 RPN 对训练 Fast R-CNN 检测网络的影响。为此，我们通过使用 2000 个 SS 提议和 ZF 网络来训练 Fast R-CNN模型。我们固定此检测器并通过更改测试时使用的提议区域来评估检测 mAP。在这些消融实验中，RPN 不与检测器共享特征。  

在测试期间用 300 个RPN提议替换 SS 导致 56.8% 的mAP。mAP 中的损失是因为训练/测试提议不一致。该结果作为以下比较的基准。  

有些令人惊讶的是，RPN 在测试时使用 top-ranked 100个提议仍然会输出有竞争性的结果（55.1%），表明排名最高的 RPN 提议是准确的。另一方面，使用排名最高的 6000 个 RPN 提议（无NMS）具有可比的 mAP（55.2%），这表明 NMS 不会损害检测 mAP 并可能减少误报。  

接下来，我们通过在测试时关闭 RPN 的 cls 和 reg 输出来分别研究其作用。当在测试时删除 cls 层（因此不使用 NMS/ranking）时，我们从中随机抽样N 个提议。mAP 几乎没有变化，N = 1000（55.8%），但当 N = 100 时显着下降到44.6%。这表明，cls 分数是最高级别提议的准确度的原因。  

另一方面，当在测试时去除 reg 层（因此提议成为 anchor boxes）时，mAP 下降到52.1%。这表明高质量的提议主要是由于回归框限制。锚箱虽然具有多种比例尺和纵横比，但不足以进行精确检测。  

我们还评估了更强大的网络对 RPN 提议质量的影响。我们使用 VGG-16 来训练 RPN，并仍然使用 SS + ZF 的上述检测器。mAP从56.8%（使用 RPN + ZF）提高到 59.2%（使用 RPN + VGG）。这是一个有希望的结果，因为它表明 RPN + VGG 的建议质量比 RPN + ZF 的建议质量好。由于 RPN+ZF 的提议与 SS 结果差不多，因此我们可能预计 RPN + VGG 优于 SS。以下实验验证了这个假设。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/5.jpg?raw=true)

**VGG-16 的性能** 表3 显示了 VGG-16 的提议和检测结果。使用 RPN + VGG，非共享特征的结果为68.5%，略高于 SS 基线。如上所示，这是因为 RPN+VGG 生成的提议比 SS 更准确。与预定义的 SS 不同，RPN 被主动训练并受益于更好的网络。对于特征共享变体，结果是 69.9% - 比 SS 基线要好，但提议几乎没有成本费用。我们进一步使用 PASCAL VOC 2007 和 2012 训练 RPN 和检测网络。该 mAP 是73.2%。图5 显示了 PASCAL VOC 2007 测试集的一些结果。在 PASCAL VOC 2012 测试集（表4）中，我们的方法具有 70.4%。表6 和表7 显示了详细的数字。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/7.jpg?raw=true)

在表5 中我们总结了整个物体检测系统的运行时间。根据不同内容，SS 需要1-2秒（平均大约1.5s），而使用 VGG-16 的 Fast R-CNN 在 2000 个 SS 建议中需要 320ms（如果在全连接层上使用 SVD，则需要 223ms）[2]）。我们的 VGG-16 系统在提议和检测中总共需要 198ms。在共享卷积特性的情况下，仅RPN 仅需要 10ms 来计算附加层。我们的区域计算也较低，这归功于较少的提议（每张图片300张）。我们的系统采用 ZF 网络，帧速率为 17 fps。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/8.jpg?raw=true)

**对超参数的敏感度** 在表8 中，我们调查了锚的设置。默认情况下，我们使用 3 个尺度和 3 个纵横比（表8中的69.9%mAP）。如果在每个位置仅使用一个锚点，则 mAP 将下降 3-4% 的相当大的余量。如果使用 3 个尺度（1个纵横比）或 3 个纵横比（1个尺度），则 mAP 更高，这表明使用多个尺寸的锚点作为回归参考是一个有效的解决方案。仅使用具有 1 个纵横比（69.8%）的 3 个尺度与在该数据集上使用 3 个尺度比例的 3 个尺度一样好，这表明检测准确率不严重依赖尺度和纵横比。但我们仍然在设计中采用这两个维度来保持我们系统的灵活性。  

在表9 中，我们比较了公式(1)中λ的不同值。默认情况下，我们使用λ= 10，这使得方程(1)中的两项在标准化之后大致相等地加权。表9 显示，当λ处于大约两个数量级（1至100）的范围内时，我们的结果仅略微受到影响（约1%）。这表明结果在很大范围内对λ不敏感。  

**Recall-to-IoU 分析** 接下来，我们用不同的 IoU 比例计算召回，值得注意的是，Recall-to-IoU 度量仅仅与最终检测精度有关。使用此指标来诊断提议方法比评估更合适。  

在图4 中，我们显示了使用 300,1000 和 2000 个提议的结果。我们与 SS 和 EB 进行比较，根据这些方法产生的置信度，N 个提议是排名前 N 的提议。图表显示，当提议数量从 2000 个降至 300 个时，RPN 方法表现优雅。这解释了为什么 RPN 在使用少至 300 个提议时具有良好的最终检测 mAP。正如我们以前分析过的，这个属性主要归因于 RPN 的 cls 术语。当提议较少时， SS 和 EB 召回降低的速度比 RPN 快。  

**一阶段检测与两阶段提议+检测** OverFeat 论文[9]提出了一种在卷积特征映射的滑动窗口上使用回归器和分类器的检测方法。OverFeat 是一阶段，特定类别的检测流水线，我们的是两阶段级联，包括类别不可知的提议和类别特定的检测。在 OverFeat 中，区域特征来自一个比例金字塔上一个长宽比的滑动窗口。这些特征用于同时确定对象的位置和类别。在 RPN 中，这些特征来自方形（3×3）滑动窗口并预测相对于具有不同比例和高宽比的锚的提议。尽管这两种方法都使用滑动窗口，但区域提议任务只是 Faster RCNN 的第一阶段 - 下游 Fast R-CNN 检测器会参照提议来修正它们。在我们级联的第二阶段，区域特征自适应地池化[1]，[2]来自更忠实地覆盖区域特征的提议框。我们相信这些特征可以带来更准确的检测结果。  

为了比较一阶段和两阶段系统，我们通过一阶段 Fast R-CNN 模拟 OverFeat 系统（从而也避开了实现细节的其他差异）。在这个系统中，“提议”是 3 个尺度（128,256,512）和 3 个纵横比（1：1,1：2,2：1）的密集滑动窗口。Fast R-CNN 训练来预测特定类别分数并从这些滑动窗口中回归框位置。由于OverFeat 系统采用图像金字塔，因此我们还使用从 5 个比例提取的卷积特征进行评估。我们使用[1]，[2]中的5个尺度。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/9.jpg?raw=true)

表10 比较了两阶段系统和两阶段系统的变体。使用 ZF 模型，一阶段系统的 mAP 为53.9%。这比两阶段系统（58.7%）低4.8%。该实验验证了级联区域提议和对象检测的有效性。在[2]，[39]中报道了类似的观察结果，其中用滑动窗代替 SS 区域提议导致两篇论文中的〜6% 的退化。我们也注意到，一阶系统比较慢，因为它更多的处理提议。  

### 4.2 Experiments on MS COCO  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/13.jpg?raw=true)

我们在 Microsoft COCO 对象检测数据集[12]上提供了更多结果。该数据集涉及 80 个对象类别。我们用训练集上的 80k 图像，验证集上的 40k 图像以及测试集上的 20k 图像进行实验。我们评估了 IoU∈[0.5：0.05：0.95]（COCO 的标准度量，简称为 mAP @ [.5，.95]）和 mAP@0.5（PASCAL VOC度量）的平均 mAP。  

我们的系统对这个数据集做了一些小的改动。我们在 8-GPU 上训练我们的模型，RPN（每GPU 1个）的有效最小批量大小为8，Fast R-CNN（每 GPU 2个）的有效小批量大小为 16。RPN 步骤和 Fast R-CNN 步骤都以 240k 次迭代进行训练，学习率为 0.003，然后 80k 次迭代后为 0.0003 。我们修改学习率（从0.003而不是0.001开始），因为小批量的大小发生了变化。对于锚点，我们使用 3 个纵横比和 4 个尺度（加上642），主要是处理这个数据集上的小对象。此外，在我们的 Fast R-CNN 步骤中，负样本被定义为在[0,0.5]区间内与 ground truth 具有最大 IoU 的提议，而不是[1]，[2]中使用的[0.1,0.5] ]。我们注意到，在 SPPnet 系统[1]中，[0.1,0.5] 中的负样本用于网络微调，但[0,0.5]中的负样本仍然在支持向量机的步骤中使用 hard-negative mining。但 Fast R-CNN 系统[2]放弃了 SVM 步骤，所以[0,0.1]中的负样本都不会被访问。使用这些[0,0.1]样本在 COCO 数据集上改进了 mAP@0.5（但对PASCAL VOC 的影响可以忽略不计）。  

其余的实施细节与 PASCAL VOC 相同。特别是，我们继续使用 300 个提议和单一尺度（s = 600）进行测试。COCO 数据集上的每个图像的测试时间仍然大约为 200 毫秒。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/10.jpg?raw=true)

在表11 中，我们首先报告 Fast R-CNN 系统[2]的结果，使用本文中的实现。我们的 Fast R-CNN 基线在测试集上有 39.3% 的mAP@0.5，高于[2]中的报告。我们猜想造成这种差距的原因主要是由于负样本的定义以及小批量大小的变化。我们还注意到，mAP @ [.5，.95]只是可比的。  

接下来我们评估我们的 Faster R-CNN 系统。使用 COCO 训练集训练，COCO 测试组中 Faster R-CNN 拥有 42.1% mAP@0.5 和 21.5% mAP @ [.5，.95]。与同样方案下的 Fast RCNN 相比，mAP@0.5 和 mAP@[.5，.95]分别提高 2.8% 和 2.2%（表11）。这表明 RPN 在提高 IoU 阈值时可以提高定位精度。使用COCO 训练集验证集训练，Faster RCNN 在 COCO 测试组中具有 42.7% 的 mAP@0.5 和 21.9% 的 mAP@[.5，.95]。图6 显示了 MS COCO test-dev 集合中的一些结果。  

**Faster R-CNN in ILSVRC & COCO 2015 competitions** 我们已经证明，由于 RPN 完全学习了通过神经网络提出区域，因此更好的特征使 Faster R-CNN 获益更多。即使将深度增加到 100 层以上，这种观察仍然有效[18]。只用 101 层残差网络（ResNet-101）代替 VGG-16，在COCO val set上 Faster R-CNN 系统将 mAP从 41.5%/21.2%（VGG-16）增加到 48.4%/27.2%（ResNet -101）。随着与 Faster RCNN 正交的其他改进，He 等人[18]在 COCO 测试开发组中获得了 55.7%/34.9% 的单模型结果和 59.0%/37.4% 的 ensemble 结果，在 COCO 2015 对象检测竞赛中获得了第一名。同样的系统[18]也在 ILSVRC 2015 对象检测竞赛中获得第一名，超过第二名的绝对 8.5%。RPN 也是 ILSVRC 2015 定位和 COCO 2015 分割比赛第一名获奖作品的基石，详情请参见[18]和[15]。  

### 4.3 From MS COCO to PASCAL VOC  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/detection/1/11.jpg?raw=true)

大规模数据对改善深度神经网络至关重要。接下来，我们调查 MS COCO 数据集如何帮助 PASCAL VOC 的检测性能。  

作为一个简单的基线，我们直接评估 PASCAL VOC 数据集上的 COCO 检测模型，无需对任何 PASCAL VOC 数据进行微调。这种评估是可能的，因为 COCO 类别是 PASCAL VOC 类别的超集。COCO 专用的类别在本实验中被忽略，softmax 层仅在 20 个类别加背景下执行。在 PASCAL VOC 2007 测试装置中，该设置下的 mAP 为76.1%（表12）。这一结果好于对 VOC07 + 12（73.2%）进行培训的结果。  

然后我们对 VOC 数据集上的 COCO 检测模型进行微调。在这个实验中，COCO 模型代替了 ImageNet 预先训练的模型（用于初始化网络权重），Faster R-CNN 系统按照 3.2 节的描述进行了微调。这样做会导致 PASCAL VOC 2007 测试组中 78.8% 的 mAP。来自 COCO 集合的额外数据增加了 5.6%的mAP。表6 显示，在 PASCAL VOC 2007 中，使用 COCO + VOC 进行训练的模型具有每个类别中最佳的 AP。在 PASCAL VOC 2012 测试集上观察到类似的改进（表12和表7）。我们注意到，获得这些强大结果的测试时间速度仍然大约为每个图像 200 毫秒。  

## 5 CONCLUSION  

我们展示了 RPN 可以生成高效和准确的区域提议。通过与下游检测网络共享卷积特征，区域提议步骤几乎没有成本。我们的方法使统一的，基于深度学习的物体检测系统能够以接近实时的帧率运行。学习到的 RPN 也提高了区域提议质量，从而提高了整体物体检测的准确性。

## 参考  

- 1 [Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks](https://arxiv.org/abs/1506.01497)